In [2]:
from bs4 import BeautifulSoup
import requests
import re
from webscraper import *
import copy

In [4]:
url = "https://fashionunited.com/i/top200"
webpage = requests.get(url)

soup = BeautifulSoup(webpage.text,'html.parser')

In [5]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width" name="viewport"/>
  <title>
   Top 200 Fashion Companies in the World - Public and Private
  </title>
  <link as="script" href="https://cdn.ampproject.org/v0.js" rel="preload"/>
  <meta content="FashionUnited" property="og:site_name"/>
  <meta content="@FashionUnited" name="twitter:site"/>
  <meta content="Top 200 Fashion Companies in the World - Public and Private" name="og:title" property="og:title"/>
  <meta content="Top 200 Fashion Companies in the World - Public and Private" name="twitter:title"/>
  <meta content="Find all data on the biggest fashion brand companies in the world on FashionUnited" name="description"/>
  <meta content="Find all data on the biggest fashion brand companies in the world on FashionUnited" property="og:description"/>
  <meta content="Find all data on the biggest fashion brand companies in the world on FashionUnited" name="twitter:description"/>
  <meta

In [27]:
result = soup.findAll("tr")
print(type(result))

list_of_fashion_companies = []

print(len(result))
for tr in result:


    for count, td in enumerate(tr.children):
        if count == 5:
            list_of_fashion_companies.append(td.string)

print(len(list_of_fashion_companies))

<class 'bs4.element.ResultSet'>
188
188


In [30]:
companies = ""
for company in list_of_fashion_companies:
    companies += f"{company}\n"

In [31]:
file = open("../data/companylist.txt","a")
file.write(companies)
file.close()

In [30]:
def scrape_annual_report(url:str):

    """
    Scrapes EDGAR For An Annual Report Using The Provided URL And Stores Data In Text File.

    parameters:
        url: the url of the annual report
    
    returns:
        parsed_annual_report: the entire html document (BeautifulSoup Object)
        parsed_parts_clean: a list of 
    """

    "Step 1. Scrape The HTML File"

    annual_report = requests.get(url, headers={"User-Agent": "bzhang14@umd.edu"})   # sends get request to URL endpoint (html file)
    parsed_annual_report = BeautifulSoup(annual_report.text, 'html.parser')         # parse the response using BeautifulSoup 

    "Step 2. Split into Parts"

    parts = [re.compile("^PART.I$"), re.compile("^PART.II$"), 
             re.compile("^PART.III$"), re.compile("^PART.IV$")]        # lists of parts for annual report
    parsed_parts = parsed_annual_report.find_all(string=parts)  # list of all elements that contain any of the parts in the list above

    # list of actual part seperating elements
    # note that the the actual seperating elements are stored in spans!
    parsed_parts_clean = [part for part in parsed_parts if part.parent.name != "a" 
                          and part.parent.parent.name != "a"]                              

    return parsed_annual_report, parsed_parts_clean

def find_elements_inbetween(element_1, element_2) -> list:

    """
    Useful Function To Find All The Elements Between Two Elements 
    """
    element_list = []      # list that stores bs4.element.Tag
    
    soup = BeautifulSoup(str(element_1), "html.parser")
    element_1_in_soup = soup.find(element_1.name)
    print(element_1)
    # print(soup.find(element_1.name))
    # print(len(list(element_1.next_siblings)))
    print(len(list(element_1.next_siblings)))

    for element in element_1.next_siblings:                 # iterate through the all of the elements that come after 
        element_list.append(copy.copy(element))             # element_1, this is in element_1.next_siblings

        if element == element_2:                            # if the second element is encountered then break out of loop
            break
    
    ## TODO: Write String Converter and HTML Parser get the elements in between!##
    
    for element in element_list[::-1]:

        element_1_in_soup.insert_after(element)
    
    return soup

def extract_parts(parsed_parts):

    """
    Extracts The Elements Between The Parts (1,2,3,4)
    """

    part1 = parsed_parts[0].parent.parent       # access the grand parent because the first parent is the tag
    part2 = parsed_parts[1].parent.parent       # that wraps the text "PART I", this is usually the <span>
    part3 = parsed_parts[2].parent.parent       # the tag that wraps the <span> is usually a <div>
    part4 = parsed_parts[3].parent.parent

    elements_part1_to_part2 = find_elements_inbetween(part1, part2)     
    elements_part2_to_part3 = find_elements_inbetween(part2, part3)
    elements_part3_to_part4 = find_elements_inbetween(part3, part4)

    return [elements_part1_to_part2, elements_part2_to_part3, elements_part3_to_part4, part4]



def extract_specific_item(elements: BeautifulSoup, part_num:int, item:re.Pattern):
    
    """
    Extract A Specific Item For A Specific Part Of The Annual Report.
    """

    items_list = elements.find_all(string=item)
    items_list_cleaned = [item_element for item_element in items_list if item_element.parent != "a"]


def find_item8(part2):
    regex_list = [re.compile("Item.8..Financial.Statements", re.I), re.compile("Item.8\\.",re.I)]
    for regex in regex_list:
        item_8 = part2.find_all(string=regex) # note ignore case
        print(item_8)
        if len(item_8) > 0:
            break
    return item_8
  
def find_income_statements(cik:str):
    
    urls = json_submission_scraper(cik, "10-K")  # get all the URLS of the 10-Ks
    
    count = 1
    for url in urls:
      
      parsed_annual_report, parsed_parts_clean = scrape_annual_report(url)
      parts_list = extract_parts(parsed_parts_clean)
      item8 = find_item8(parts_list[1])
      print(item8[0].parent)
      xbrl_dictionary , metric_values, statement_name_list = income_statement(item8[0].parent.parent.parent)
      
      xbrl_dictionary_string = ""
      for key,value in xbrl_dictionary.items():
         xbrl_dictionary_string += f"{key}:{value}\n"

      with open(f"../data/company data/LULU/annual report {count}.txt", "w+") as file:
        file.write(xbrl_dictionary_string)
    
      count += 1
          

In [31]:
find_income_statements("1397187")

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
['https://www.sec.gov/Archives/edgar/data/1397187/000139718723000012/lulu-20230129.htm'
 'https://www.sec.gov/Archives/edgar/data/1397187/000139718722000014/lulu-20220130.htm'
 'https://www.sec.gov/Archives/edgar/data/1397187/000139718721000009/lulu-20210131.htm'
 'https://www.sec.gov/Archives/edgar/data/1397187/000139718720000012/lulu-20200202x10k.htm'
 'https://www.sec.gov/Archives/edgar/data/1397187/000139718719000011/lulu-20190203x10k.htm'
 'https://www.sec.gov/Archives/edgar/data/1397187/000139718718000013/lulu-20180128x10k.htm'
 'https://www.sec.gov/Archives/edgar/data/1397187/000139718717000008/lulu-20170129x10k.htm'
 'https://www.sec.gov/Archives/edgar/data/1397187/000139718716000089/lulu-20160131x10k.htm'
 'https://www.sec.gov/Archives/edgar/data/1397187/000139718715000016/lulu-20150201x10k.htm'
 'https://www.sec.gov/Archives/edgar/data/1397187/000139718714000021/lulu-20140202x10k.htm'
 'https://www.sec.gov/Archives/edgar/data/13

IndexError: list index out of range

In [1]:
import re

In [32]:
regex = re.compile("^\(?[0-9]{1,3}(,[0-9]{3})*\)?$")
string = "12,320,250"
match_object = re.search(regex,string)

print(match_object)
match_object.group()

<re.Match object; span=(0, 10), match='12,320,250'>


'12,320,250'